### LSTM Assignement John Isak Villanger
For this assignement I used the meditations by Marcus Aurelius as data.
Here is some of the generated wisdom:

![Alt text](https://imagizer.imageshack.com/img922/5897/nYX4lW.jpg?raw=true "Title")

![Alt text](https://imagizer.imageshack.com/img923/888/I5yxZo.jpg?raw=true "Title")

![Alt text](https://imagizer.imageshack.com/img922/6212/ehyEsS.jpg?raw=true "Title")




In [1]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F
import matplotlib.pyplot as plt
import copy
import requests
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#with urlopen('https://www.gutenberg.org/files/11/11-0.txt') as file:
#    text = file.read()
text = ""
for i in range(11, 100):
    response = requests.get("https://www.gutenberg.org/files/" + str(i) + "/" + str(i) + "-0.txt")
    response = response.text
    text += response
#print(text[0:1000])
text = re.sub("\d\.","", text) # Replace the leading before each chapter
text = re.sub("[\t\r]"," ", text) # Replace newlines
text = re.sub("[^A-z,. \n]","", text) # Remove everything that's not a normal char ,. or space
text = re.sub("  *"," ", text) # Replace instances of two or more spaces with one

print(text[0:1000])
#Code taken from: https://towardsdatascience.com/generating-text-using-a-recurrent-neural-network-1c3bfee27a5e
chars = sorted(list(set(text))) # Getting all unique chars
print('total chars: ', len(chars))
print(chars)
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
#print(text)


maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])


# One Hot encoding of data
x = torch.zeros((len(sentences), maxlen, len(chars)), dtype=torch.float32)
y = torch.zeros((len(sentences)), dtype=torch.long)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i] = char_indices[next_chars[i]]


torch.save(x,"100booksX.pt")
#print(x)
torch.save(y,"100booksY.pt")
#x = torch.load("x.pt")
#y = torch.load("y.pt")

The Project Gutenberg eBook of Alices Adventures in Wonderland, by Lewis Carroll 
 
This eBook is for the use of anyone anywhere in the United States and 
most other parts of the world at no cost and with almost no restrictions 
whatsoever. You may copy it, give it away or reuse it under the terms 
of the Project Gutenberg License included with this eBook or online at 
www.gutenberg.org. If you are not located in the United States, you 
will have to check the laws of the country where you are located before 
using this eBook. 
 
Title Alices Adventures in Wonderland 
 
Author Lewis Carroll 
 
Release Date January, [eBook ] 
[Most recently updated October , ] 
 
Language English 
 
Character set encoding UTF 
 
Produced by Arthur DiBianca and David Widger 
 
 START OF THE PROJECT GUTENBERG EBOOK ALICES ADVENTURES IN WONDERLAND 
 
[Illustration] 
 
 
 
 
Alices Adventures in Wonderland 
 
by Lewis Carroll 
 
THE MILLENNIUM FULCRUM EDITION 
 
Contents 
 
 CHAPTER I. Down the RabbitHole 
 

RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 62782192000 bytes. Buy new RAM!

In [3]:
class DataSet(torch.utils.data.Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y
    if len(self.X) != len(self.Y):
      raise Exception("The length of X does not match the length of Y")

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    # note that this isn't randomly selecting. It's a simple get a single item that represents an x and y
    _x = self.X[index]
    _y = self.Y[index]

    return _x, _y

In [4]:
data = DataSet(x, y)
print(len(data))
train, val = torch.utils.data.random_split(data, [73500, 1022], generator=torch.Generator().manual_seed(42))
train = torch.utils.data.DataLoader(train, 256, shuffle=True)
val = torch.utils.data.DataLoader(val, 256, shuffle=False)

NameError: name 'x' is not defined

In [ ]:
class LSTMPredictor(nn.Module):
    def __init__(self, n_hidden, n_layers):
        super(LSTMPredictor, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.lstm = nn.LSTM(len(chars), n_hidden, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(n_hidden, len(chars))
        self.softmax = nn.Softmax()

    def forward(self, x, hidden):   # -> input x:    (batch_size, seq_length, num_features)
        x, hidden = self.lstm(x, hidden) # -> LSTM out:   (batch_size, seq_length, hidden_size)
        x = x[:,-1]  # -> Taking last output from each sequence.
        x = self.linear(x)
        return x, hidden


In [ ]:
model = LSTMPredictor(n_hidden=512, n_layers=2)
model = model.to(device)
train_size = 73000
val_size = 1164
adam = torch.optim.Adam(model.parameters(), lr=0.001)
cross_entropy = nn.CrossEntropyLoss()


# Will automatically restore weights from epoch with best validation accuracy
def train_model(model, loss_f, optimizer, num_epochs):
    best_model_weights = copy.deepcopy(model.state_dict())
    best_model_acc = 0.0
    train_loss = np.zeros(num_epochs)
    train_acc = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)
    val_acc = np.zeros(num_epochs)
    for epoch in range(num_epochs):
        print("epoch",epoch+1)
        running_loss = 0.0
        running_corrects = 0
        torch.set_grad_enabled(True)
        best_acc = 0
        model.train()
        for batch_idx, batch in enumerate(train):
            model.zero_grad()
            #print(batch.shape)
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            #Forward
            out, hidden = model(inputs, None )
            #print(out)
            _, preds = torch.max(out, 1)
            #print(preds)
            #print(type(preds[0]))

            #Compute objective function
            #print(out)
            #print(labels)
            loss = loss_f(out, labels)

            #Clean the gradients
            #model.zero_grad()

            #Accumulate partial deriviates wrt parameters
            loss.backward()

            #Step in the opposite direction og the gradient wrt optimizer
            optimizer.step()

            #stats
            running_loss += loss.item() * inputs.size(0)
            #print(preds)
            #print(labels.data)
            running_corrects += torch.sum(preds == labels.data)

        train_loss[epoch] = running_loss / train_size
        train_acc[epoch] = running_corrects.double() / train_size
        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss[epoch], train_acc[epoch]))
        running_loss = 0.0
        running_corrects = 0
        for batch_idx, batch in enumerate(val):
            model.eval()
            inputs, labels = batch

            #Move to gpu if availible
            inputs, labels = inputs.to(device), labels.to(device)
            #Forward
            with(torch.no_grad()):
                out, hidden = model(inputs, None)
            _, preds = torch.max(out,1)

            #Compute objective function
            loss = loss_f(out, labels)


            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)


        val_loss[epoch] = running_loss / val_size
        val_acc[epoch] = running_corrects.double() / val_size

        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss[epoch], val_acc[epoch]))

        # deep copy the model
        if (val_acc[epoch] > best_acc):
            best_acc = val_acc[epoch]
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model, train_acc, train_loss, val_acc, val_loss




model, train_acc, train_loss, val_acc, val_loss = train_model(model, cross_entropy, adam, 15)
torch.save(model, "model")
#Plotting the data
fig, ax = plt.subplots()
ax.plot(train_acc, label="Training")
ax.plot(val_acc, label="Validation")
ax.set_xticklabels(np.arange(len(train_acc)))
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Training-validation accuracy")
plt.legend()
plt.show()


fig, ax = plt.subplots()
ax.plot(train_loss, label="Training")
ax.plot(val_loss, label="Validation")
ax.set_xticklabels(np.arange(len(train_loss)))
plt.xlabel("Epoch-1")
plt.ylabel("Loss")
plt.title("Training-validation loss")
plt.legend()
plt.show()

In [ ]:
#model = torch.load("modeltest")
model.eval()
model.zero_grad()
def Probabilistic_selection(out):
    out = F.F.softmax(out, 1)
    selection = np.random.uniform()
    rolling_sum = 0
    index = 0
    while True:
            rolling_sum += out[0][index]
            if rolling_sum > selection:
                break
            index += 1
    preds =[index]
    return preds

for j in range(1):
    seed = "The purpose of life"
    answer = ""

    #Generates a random seed of 3 letters
    #for k in range(3):
    #    seed += chr(np.random.randint(97, 122))
    x = torch.zeros((len(seed), len(chars)), dtype=torch.float32)
    for i in range(len(seed)):
        x[i, char_indices[seed[i]]] = 1

    hidden = None
    x = x.view(-1, len(x), len(x[0]))
    x = x.to(device)
    for i in range(1000):

        out, hidden = model(x, hidden)
        #preds = Probabilistic_selection(out) #Selects letters probabilistically from a softmax activation
        _, preds = torch.max(out,1) #Selects letters deterministically
        char = indices_char[int(preds[0])]
        answer += char
        x = torch.zeros((1,1,len(chars)), dtype=torch.float32, device=device)
        x[0,0,int(preds[0])] = 1

    print("\n" + answer)

